In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.utils import class_weight, shuffle

from keras import applications
from keras import optimizers
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [3]:
foldernames = os.listdir('animals/raw-img')
categories = []
files = []
i = 0
for k, folder in enumerate(foldernames):
    filenames = os.listdir("animals/raw-img/" + folder)
    for file in filenames:
        files.append("animals/raw-img/" + folder + "/" + file)
        categories.append(k)
        
df = pd.DataFrame({
    'filename': files,
    'category': categories
})
df.head()

filename  category
0  animals/raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOW...         0
1  animals/raw-img/cane/OIP---A27bIBcUgX1qkbpZOPs...         0
2  animals/raw-img/cane/OIP---cByAiEbIxIAleGo9AqO...         0
3  animals/raw-img/cane/OIP---ZIdwfUcJeVxnh47zppc...         0
4  animals/raw-img/cane/OIP---ZRsOF7zsMqhW30WeF8-...         0

In [5]:
train_df = pd.DataFrame(columns=['filename', 'category'])
for i in range(10):
    subset_df = df[df.category == i].iloc[:500,:]  # Lấy 500 bản ghi từ mỗi category
    train_df = pd.concat([train_df, subset_df], ignore_index=True)  # Kết hợp các DataFrame

train_df.head()
train_df = train_df.reset_index(drop=True)
train_df

filename category
0     animals/raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOW...        0
1     animals/raw-img/cane/OIP---A27bIBcUgX1qkbpZOPs...        0
2     animals/raw-img/cane/OIP---cByAiEbIxIAleGo9AqO...        0
3     animals/raw-img/cane/OIP---ZIdwfUcJeVxnh47zppc...        0
4     animals/raw-img/cane/OIP---ZRsOF7zsMqhW30WeF8-...        0
...                                                 ...      ...
4995  animals/raw-img/scoiattolo/OIP-DLuAMgbGGOGW0r4...        9
4996  animals/raw-img/scoiattolo/OIP-DlVY-0x9uqexVnZ...        9
4997  animals/raw-img/scoiattolo/OIP-DlwICIPtlrWDrdz...        9
4998  animals/raw-img/scoiattolo/OIP-DL_IigTXGyMfa6O...        9
4999  animals/raw-img/scoiattolo/OIP-dMd2HnsRjgUsYCA...        9

[5000 rows x 2 columns]

In [6]:
y = train_df['category']
x = train_df['filename']
y = train_df['category']

x, y = shuffle(x, y, random_state=8)

In [8]:
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

images = []
with tqdm(total=len(train_df)) as pbar:
    for i, file_path in enumerate(train_df.filename.values):
        #read image
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #resize
        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        #centering
        img = centering_image(cv2.resize(img, dsize=tile_size))

        #out put 224*224px 
        img = img[16:240, 16:240]
        images.append(img)
        pbar.update(1)

images = np.array(images)

C:\Users\Tienn Vo\AppData\Local\Temp\ipykernel_11016\1626648720.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=len(train_df)) as pbar:


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [11]:
src_directory = dataset_folder
for file in listdir(src_directory):
    if file.startswith('cat'):
        cat_files.append(file)
    elif file.startswith('dog'):
        dog_files.append(file)

In [12]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Tiền xử lý ảnh cho VGG16 (preprocess_input chuẩn từ ImageNet)
from tensorflow.keras.applications.vgg16 import preprocess_input
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

MemoryError: Unable to allocate 16.3 GiB for an array with shape (29075, 224, 224, 3) and data type float32

In [ ]:
# Khởi tạo ImageDataGenerator với các kỹ thuật tăng cường dữ liệu (augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Chuẩn hóa giá trị pixel
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Tải dữ liệu từ thư mục và phân loại theo nhãn (y)
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [ ]:
# Tải mô hình VGG16 đã được huấn luyện trước (ImageNet), không có phần phân loại cuối
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
# Đóng băng các lớp của mô hình cơ bản (không huấn luyện lại)
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Thêm các lớp phân loại của riêng bạn
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Chuyển đổi ra vector đặc trưng
x = Dropout(0.5)(x)  # Dropout để tránh overfitting
x = Dense(256, activation='relu')(x)  # Lớp fully connected
output = Dense(len(CATEGORIES), activation='softmax')(x)  # Lớp phân loại cuối cùng


In [ ]:
# Tạo mô hình
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Biên dịch mô hình
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Hiển thị mô hình
model.summary()

In [ ]:
# Huấn luyện mô hình
model.fit(train_generator, epochs=20, validation_data=val_generator)


In [ ]:
model.save("vgg16_animal_classifier.h5")